In [57]:
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

warnings.filterwarnings('ignore')

In [58]:
df = pd.read_csv('heart.csv')
df_copy = df.copy()

In [59]:
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [60]:
df.isnull().sum() + df.isna().sum()

Age               0
Sex               0
ChestPainType     0
RestingBP         0
Cholesterol       0
FastingBS         0
RestingECG        0
MaxHR             0
ExerciseAngina    0
Oldpeak           0
ST_Slope          0
HeartDisease      0
dtype: int64

In [61]:
def one_hot_encoding(data_frame,column_name, prefix_name):
    temp = pd.get_dummies(data_frame[column_name], prefix=prefix_name) * 1
    data_frame.drop(column_name, axis=1, inplace=True)
    return pd.concat([data_frame, temp], axis=1)

In [63]:
columns_to_perform_one_hot_encoding = {
    'ChestPainType': 'Cheest_Pain',
    'RestingECG': 'Resting_ECG',
    'ExerciseAngina': 'Exercise_Angina',
    'ST_Slope': 'ST_Slope',
    'Sex': 'Sex'
}
for key in columns_to_perform_one_hot_encoding:
    df = one_hot_encoding(df, key, columns_to_perform_one_hot_encoding[key])


In [64]:
df.head()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease,Cheest_Pain_ASY,Cheest_Pain_ATA,Cheest_Pain_NAP,...,Resting_ECG_LVH,Resting_ECG_Normal,Resting_ECG_ST,Exercise_Angina_N,Exercise_Angina_Y,ST_Slope_Down,ST_Slope_Flat,ST_Slope_Up,Sex_F,Sex_M
0,40,140,289,0,172,0.0,0,0,1,0,...,0,1,0,1,0,0,0,1,0,1
1,49,160,180,0,156,1.0,1,0,0,1,...,0,1,0,1,0,0,1,0,1,0
2,37,130,283,0,98,0.0,0,0,1,0,...,0,0,1,1,0,0,0,1,0,1
3,48,138,214,0,108,1.5,1,1,0,0,...,0,1,0,0,1,0,1,0,1,0
4,54,150,195,0,122,0.0,0,0,0,1,...,0,1,0,1,0,0,0,1,0,1


In [65]:
# Assuming X is your feature matrix
numerical_features = ['Age', 'RestingBP', 'Cholesterol', 'MaxHR', 'Oldpeak']
scaler = StandardScaler()
df[numerical_features] = scaler.fit_transform(df[numerical_features])

In [66]:
df.head()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease,Cheest_Pain_ASY,Cheest_Pain_ATA,Cheest_Pain_NAP,...,Resting_ECG_LVH,Resting_ECG_Normal,Resting_ECG_ST,Exercise_Angina_N,Exercise_Angina_Y,ST_Slope_Down,ST_Slope_Flat,ST_Slope_Up,Sex_F,Sex_M
0,-1.433140,0.410909,0.825070,0,1.382928,-0.832432,0,0,1,0,...,0,1,0,1,0,0,0,1,0,1
1,-0.478484,1.491752,-0.171961,0,0.754157,0.105664,1,0,0,1,...,0,1,0,1,0,0,1,0,1,0
2,-1.751359,-0.129513,0.770188,0,-1.525138,-0.832432,0,0,1,0,...,0,0,1,1,0,0,0,1,0,1
3,-0.584556,0.302825,0.139040,0,-1.132156,0.574711,1,1,0,0,...,0,1,0,0,1,0,1,0,1,0
4,0.051881,0.951331,-0.034755,0,-0.581981,-0.832432,0,0,0,1,...,0,1,0,1,0,0,0,1,0,1
